# 1. Загрузка и подготовка данных

## Данные

В наличии были следующие данные о месторождениях:

- признаки точек 
- объём запасов в скважине

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from scipy import stats as st

# данные
region0 = pd.read_csv('/datasets/geo_data_0.csv')
region1 = pd.read_csv('/datasets/geo_data_1.csv')
region2 = pd.read_csv('/datasets/geo_data_2.csv')

# дропнем Id, тк по нему нельзя рассчитать
region0 = region0.drop(['id'], axis = 1)
region1 = region1.drop(['id'], axis = 1)
region2 = region2.drop(['id'], axis = 1)

# переведем столбец product в баррели(умножим на 1000, тк изначально значения в тыс.барр.)
region0['product']=region0['product']*1000
region1['product']=region1['product']*1000
region2['product']=region2['product']*1000





Данные загружены. Убран столбец id, столбец product переведен в баррели( умножен на 1000).

# 2. Обучение и проверка модели

In [2]:
def target_and_pred(data):
    data_train, data_valid = train_test_split(data, test_size=0.25, random_state=12345)
    features_train = data_train.drop(['product'], axis=1)
    target_train = data_train['product']
    features_valid = data_valid.drop(['product'], axis=1)
    target_valid = data_valid['product']
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    return target_valid, predicted_valid
    

target0, predicted0 = target_and_pred(region0)
target1, predicted1 = target_and_pred(region1)
target2, predicted2 = target_and_pred(region2) 


# переведём из numpy array в Series
predicted0=pd.Series(predicted0)
predicted1=pd.Series(predicted1)
predicted2=pd.Series(predicted2)

Функция для выделения целевого признака и предказаний из валидационных выборок. Для дальнейшего удобства предсказания переведены в Series.

In [3]:
def mean_and_rmse(target, predicted):   
    mse = mean_squared_error(target, predicted)
    mean_product = target.mean()
    rmse = mse ** 0.5
    return mean_product, rmse

Функция для оценки среднего запаса сырья и RMSE.

In [4]:
mean_product0, rmse0 = mean_and_rmse(target0, predicted0)
mean_product1, rmse1 = mean_and_rmse(target1, predicted1)
mean_product2, rmse2 = mean_and_rmse(target2, predicted2)
print('Средний запас сырья в регионе 0:',mean_product0, 'Rmse в регионе 0:',rmse0)
print('Средний запас сырья в регионе 1:',mean_product1, 'Rmse в регионе 1:',rmse1)
print('Средний запас сырья в регионе 2:',mean_product2, 'Rmse в регионе 2:',rmse2)

Средний запас сырья в регионе 0: 92078.59674082928 Rmse в регионе 0: 37579.421715081306
Средний запас сырья в регионе 1: 68723.13602435999 Rmse в регионе 1: 893.0992867756171
Средний запас сырья в регионе 2: 94884.23280885436 Rmse в регионе 2: 40029.70873393434


## Вывод.  У региона1 : наименьший средний запас сырья, но и наименьший разброс -> значит там не будет очень больших, и очень маленьких запасов. Предварительно выбираем между 0 и 2, но нужен дальнейший анализ.

# 3. Подготовка к расчёту прибыли

In [5]:
#константы
spots = 500 # кол-во точек для разведки
budget = 10000000000 # общий бюджет на регион
drill_cost = 50000000 # стоимость бурения 1 скважины
barrel_cost = 4500 # цена за 1 баррель
risk_threshold = 0.025 # граница риска


Константы загружены.

In [6]:
#cредний объем сырья - объем, при котором операционная прибыль равна 0
min_mean_product = drill_cost / barrel_cost
print(min_mean_product)


11111.111111111111


У всех регионов средний запас сырья больше минимального для безубыточной добычи. На этом этапе никакой регион исключить нельзя. Проводим дальнейший анализ.

In [7]:
def revenue(target,probs,count):
    probs_sorted = probs.sort_values(ascending=False) # сортируем предсказания
    selected = target[probs_sorted.index][:count] # выбираем определенное количество 
    return barrel_cost * selected.sum() # умножаем количество на цену барреля 


Функция для подсчёта прибыли. 


# 4. Расчёт прибыли и рисков 

In [8]:
state = np.random.RandomState(12345) # состояние генератора сл.чисел
 
def ci_and_mean(target,probs):   
    values = []
    for i in range(1000):
        target_subsample = target.sample(n=spots, replace=True, random_state=state)
        probs_subsample = probs[target_subsample.index]
        values.append(revenue(target_subsample,probs_subsample,200)) # 200 = budget/drill_cost
        
    values = pd.Series(values)
    mean = values.mean()
    confidence_interval =  st.t.interval(0.95, len(values)-1, values.mean(),values.sem())
    confidence_interval = (round(confidence_interval[0]/1000000000, 3), round(confidence_interval[1]/1000000000, 3))
    values_best = values.sort_values(ascending=False).head(200) # выберем 200 лучших месторождений
    values_worst=values_best.where(values_best < drill_cost).dropna() # из них выберем точки, прибыль с которых меньше стоимости бурения
    risk = values_worst.size / values_best.size * 100 # подсчитаем долю "плохих" месторождений 
    
    print('Средняя прибыль:', round(mean/1000000000, 3),'млрд. руб.')
    print('Доверительный интервал:', confidence_interval,'млрд. руб.')
    print('Риск:', risk, '%')


Функция для подсчёта средней прибыли, 95% доверительного интервала и риска.

In [9]:
ci_and_mean(target0, predicted0)

/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:1143: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


Средняя прибыль: 82.729 млрд. руб.
Доверительный интервал: (82.549, 82.909) млрд. руб.
Риск: 0.0 %


In [10]:
ci_and_mean(target1, predicted1)

Средняя прибыль: 61.885 млрд. руб.
Доверительный интервал: (61.701, 62.069) млрд. руб.
Риск: 0.0 %


In [11]:
ci_and_mean(target2, predicted2)


Средняя прибыль: 85.196 млрд. руб.
Доверительный интервал: (85.011, 85.38) млрд. руб.
Риск: 0.0 %


## Вывод: Доля месторождений, где прибыль меньше стоимости бурения, равна 0 -> все регионы безубыточны, рисков нет. Тк мы выбираем между регионом0 и регионом2 - выбираем регион 2. У него больше средняя прибыль(85 против 83 млрд. руб.) и границы 95% доверительного интервала тоже больше.


# Общий вывод. Целевой регион - регион2. У него самый большой средний запас сырья на месторождение(94884 баррелей), больше средняя прибыль с месторождения(85 млрд. руб.).